# 오차역전파법 (BackPropagation)
- 가중치 매개변수에 대한 손실함수의 기울기를 수치미분보다 더 효율적으로 구하는 알고리즘.
- 수식을 이용한 설명, 계산 그래프를 이용한 설명 두 가지가 있다. 책에서는 계산 그래프를 이용해 설명함.

## 계산 그래프 (computational graph)
- 계산 과정을 그래프로 나타낸 것. 여기서 그래프는 자료구조에 나오는 그래프.
- node에 연산자를 적고, edge위에 계산할 수(중간 연산 결과)를 적는다. 
- 계산 방향은 왼쪽에서 오른쪽.
- 이것이 바로 순전파. 
- 연산 결과의 미분값을 반대 방향으로 전달하면 역전파.

### 계산 그래프 특징
- 순전파
  - 국소적 계산을 전파하여 최종적 결과를 얻는다.
    - 복잡한 문제를 단순한 계산의 결합으로 풀 수 있다.
- 역전파
  - 국소적 미분을 전달하는 과정에서 중간 미분 결과를 공유할 수 있어 다수의 미분을 효율적으로 구할 수 있다.


In [1]:
import numpy as np

In [2]:
# 곱셈 계층 구현
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y

    return out

  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

In [3]:
# 사과 판매액 계산
apple_price = 100
amount = 2
tax = 1.1

# 계층 object 생성
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
sum_price = mul_apple_layer.forward(apple_price, amount)
final_price = mul_tax_layer.forward(sum_price, tax)

print(final_price)

220.00000000000003


In [4]:
# 역전파 계산
dfinal_price = 1
dsum_price, dtax = mul_tax_layer.backward(dfinal_price)
dapple_price, damount = mul_apple_layer.backward(dsum_price)
print(dtax, dapple_price, damount)

200 2.2 110.00000000000001


In [5]:
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [6]:
# 초기값 설정
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층 object 생성
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
sum_apple_orange = add_apple_orange_layer.forward(apple_price, orange_price)
final = mul_tax_layer.forward(sum_apple_orange, tax)

print(final)

715.0000000000001


In [7]:
# 역전파
dfinal = 1
dsum_apple_orange, dtax = mul_tax_layer.backward(dfinal)
dapple_price, dorange_price = add_apple_orange_layer.backward(dsum_apple_orange)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(dapple, dapple_num, dorange, dorange_num, dtax)

2.2 110.00000000000001 3.3000000000000003 165.0 650


In [8]:
# relu 계층 구현
# x > 0 then (round)L/(round)x = 1, else : 0
class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, x): # x가 ndarray로 온다고 가정.
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0

    return out
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    
    return dx

In [9]:
# mask example
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)
x[mask] = 0
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
[[1. 0.]
 [0. 3.]]


In [10]:
# sigmoid layer 구현
# 1 / (1 + exp(-x)) 를 계산 그래프로 그려 나타낸 후, 역전파를 구하면 최종적으로 y(1-y) 를 곱하면 됨.
class Sigmoid:
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    
    return dx

In [11]:
x = np.random.rand(2,3)
x

array([[0.81092274, 0.82136407, 0.23704875],
       [0.84166261, 0.50690465, 0.1390751 ]])

In [12]:
x.shape

(2, 3)

In [13]:
x = np.random.rand(2)
x

array([0.07325022, 0.06870671])

In [14]:
x.shape

(2,)

In [15]:
class Affine: # 입력 데이터 x가 2차원이라고 가정. 책에서는 x를 강제로 2차원으로 reshape함.
  def __init__(self, w, b):
    self.w = w
    self.b = b
    self.x = None
    self.dw = None
    self.db = None
  
  def forward(self, x):
    self.x = x
    out = np.dot(self.x, self.w) + self.b

    return out
  
  def backward(self, dout):
    dx = np.dot(dout, self.w.T)
    self.dw = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    return dx

In [16]:
def softmax(x):
  c = np.max(x)
  exp_x = np.exp(x-c)
  sum_exp_x = np.sum(exp_x)
  y = exp_x / sum_exp_x

  return y

In [17]:
def cross_entropy_error(y, t):
  delta = 1e-7
  if y.ndim == 1:
    y = y.reshape(1, y.size)
    t = t.reshape(1, t.size)

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + delta)) / batch_size

In [18]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y= None
    self.t = None

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)

    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size

    return dx

# 오차역전파를 사용하는 신경망 구현
- 책 ch05/two_layer_net.py 참고.

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
cd /content/drive/MyDrive/Colab Notebooks/ML-Study/ML-Study/DeepLearning/DeepLearning_from_Scratch/ch04. train

/content/drive/MyDrive/Colab Notebooks/ML-Study/ML-Study/DeepLearning/DeepLearning_from_Scratch/ch04. train


In [21]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import import_ipynb
from numerical_diff import numerical_gradient_general
from collections import OrderedDict

In [24]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size,
               weight_init_std=0.01):
    self.params = {}
    self.params['w1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['w2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['w1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['w2'], self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()

  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)

    return x

  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1:
      t = np.argmax(t, axis=1)
    
    accuracy = np.sum(y==t) / float(x.shape[0])

    return accuracy

  def numerical_gradient(self, x, t):
    loss_w = lambda w : self.loss(x, t)

    grads = {}
    grads['w1'] = numerical_gradient_general(loss_w, self.params['w1'])
    grads['b1'] = numerical_gradient_general(loss_w, self.params['b1'])
    grads['w2'] = numerical_gradient_general(loss_w, self.params['w2'])
    grads['b2'] = numerical_gradient_general(loss_w, self.params['b2'])

    return grads

  def gradient(self, x, t):
    # 순전파
    self.loss(x, t)

    # 역전파
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    # 결과 저장
    grads = {}
    grads['w1'] = self.layers['Affine1'].dw
    grads['b1'] = self.layers['Affine1'].db
    grads['w2'] = self.layers['Affine2'].dw
    grads['b2'] = self.layers['Affine2'].db

    return grads

In [25]:
cd /content/drive/MyDrive/Colab Notebooks/ML-Study/ML-Study/DeepLearning/dataset

/content/drive/MyDrive/Colab Notebooks/ML-Study/ML-Study/DeepLearning/dataset


In [26]:
from mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
  diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
  print(key + ":" + str(diff))

w1:0.00012851625382384843
b1:0.0009881256155063179
w2:0.0028280701437851477
b2:0.06666666678657075


오차가 좀 커보이는데 원인은 잘 모르겠다.    
책에서는 소수점 이하 13번째까지도 가는데 내꺼는 왜 이럴까.

In [27]:
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# hyper-parameters
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
input_size = x_train.shape[1]
output_size = t_train.shape[1]

# 1 epoch 당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=input_size, hidden_size=50, output_size=output_size)

for i in range(iters_num):
  # 미니배치 얻기
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # calculate gradient 
  # grad = network.numerical_gradient(x_batch, t_batch) # 너무 오래 걸려....

  # calculate gradient by backpropagation
  grad = network.gradient(x_batch, t_batch)

  # update parameters
  for key in ('w1', 'b1', 'w2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
  
  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  # 1 epoch 당 정확도 계산
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train_acc, test_acc | " + str(train_acc) + ", " + str(test_acc))

train_acc, test_acc | 0.1118, 0.1144
train_acc, test_acc | 0.0993, 0.1032
train_acc, test_acc | 0.09863333333333334, 0.0958
train_acc, test_acc | 0.09751666666666667, 0.0974


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in subtract
  This is separate from the ipykernel package so we can avoid doing imports until


train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098
train_acc, test_acc | 0.09871666666666666, 0.098


빠르니까 좋다.. 근데 왜 정확도 개선이 안 되지?